# Space

In [ ]:
import os
import sys 
import pandas as pd 
from IPython.display import display, HTML
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)
import sys
from proj_space import PROJECT, TaskName, SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
recfldtkn_config_path = os.path.join(SPACE['CODE_RFT'], 'config_recfldtkn')

# Load dfHumanRecAttr

In [ ]:
###########################
FldTknName = 'RecName-FldTkn'# <-------- select your yaml file name
###########################

RecName = FldTknName.split('-')[0]
print(RecName)

In [ ]:
import pandas as pd
from recfldtkn.configfn import load_cohort_args, load_record_args, load_fldtkn_args

# step 1: create the FldTkn yaml file in recfldtkn_config_path
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
record_args = load_record_args(RecName, cohort_args)

# Create a HTML link and display it
path = record_args['yaml_file_path']
full_path = os.path.join(WORKSPACE_PATH, path)
display(HTML(f'{path} <a href="{full_path}" target="_blank">Open File</a>'))

In [ ]:
from recfldtkn.loadtools import load_ds_rec_and_info
# load fldtkn_args
fldtkn_args = load_fldtkn_args(RecName, FldTknName, cohort_args)

# load dfHumanRecAttr
value_cols = fldtkn_args['value_cols']
attr_cols  = fldtkn_args['attr_cols'] # record_args['RecIDChain'] + value_cols
dsHumanRecAttr, _ = load_ds_rec_and_info(RecName, cohort_args)
dfHumanRecAttr = dsHumanRecAttr.select_columns(attr_cols).to_pandas()
print(dfHumanRecAttr.shape)
dfHumanRecAttr.head()

# [Pre-defined Token Vocab]

In [ ]:
print(fldtkn_args['attr_cols'])

############################### for Cate Tkn only
column_to_top_values = {}
# TOP_NUM = 30
# cols = fldtkn_args['value_cols']
# for col in cols:
#     top_tkn = list(dfHumanRecAttr[col].value_counts().iloc[:TOP_NUM].index)
#     print(col, len(top_tkn), top_tkn)
#     column_to_top_values[col] = top_tkn # tolist()
fldtkn_args[f'column_to_top_values'] = column_to_top_values
###############################


# ##################### N2C only
item_to_configs = {} # for N2C only, if not N2C, you can leave it empty.
# print(fldtkn_args['value_cols'])
# cols = fldtkn_args['value_cols']
# descp = dfHumanRecAttr[cols].astype(float).describe().round(2)#.to_dict()
# print(descp)
# item_to_configs = {
#     'XXX': {'Max': 600, 'Min': 1, 'INTERVAL': 1},
# }
fldtkn_args['item_to_configs'] = item_to_configs
# #####################


# [Tokenizer]

In [ ]:
import inspect

################################## You might need to change it a bit. 
def tokenizer_fn(rec, fldtkn_args):
    column_to_top_values = fldtkn_args[f'column_to_top_values']
    
    d = {}
    for key in column_to_top_values:
        top_values = column_to_top_values[key]
        value = rec.get(key, 'unk')
        if value not in top_values and value != 'unk': value = 'minor'
        key_value = f"{key}_{value}"  # Concatenate key and value
        d[key_value] = 1

    tkn = list(d.keys())
    wgt = list(d.values())
    output = {'tkn': tkn, 'wgt': wgt}
    return output
##################################

tokenizer_fn.fn_string = inspect.getsource(tokenizer_fn)

print('show tokenizer_fn result')
rec = dfHumanRecAttr.iloc[0]
print(rec.to_dict())
print(tokenizer_fn(rec, fldtkn_args))

# [Vocab]

In [ ]:
############################################## Cate Tkn only
# idx2tkn = []
# for col, values in column_to_top_values.items():
#     idx2tkn = idx2tkn + [f'{col}_unk', f'{col}_minor']
#     for val in values:
#         idx2tkn.append(f"{col}_{val}")
# print(len(idx2tkn))
# print(idx2tkn[:10])
##############################################

############################################## for N2C only
# import itertools
# Min, Max = -10, 3000 # <---- keep this as default. don't change it.
# df_simu = pd.DataFrame({
#     col: [None] + list(range(Min, Max )) for col in item_to_configs.keys()
# })
# df_sim = pd.DataFrame(df_simu.apply(lambda rec: tokenizer_fn(rec, fldtkn_args), axis = 1).to_list())
# idx2tkn = sorted(list(set(itertools.chain(*df_sim['tkn'].to_list()))))
# print(len(idx2tkn))
# print(idx2tkn[:10])
##############################################

############################################## for Nume
# idx2tkn = fldtkn_args['value_cols'] + [f'{col}_None' for col in fldtkn_args['value_cols']]
# print(len(idx2tkn))
# print(idx2tkn[:10])
##############################################

In [ ]:
from recfldtkn.pipeline_record import get_and_save_vocab_from_idx2tkn
Vocab = get_and_save_vocab_from_idx2tkn(idx2tkn, **fldtkn_args)
Vocab

# Save and Load PyFile

In [ ]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
iterative_variables = [ column_to_top_values, idx2tkn ]
fn_variables = [tokenizer_fn]
pycode = convert_variables_to_pystirng(iterative_variables = iterative_variables, fn_variables = fn_variables, prefix = prefix)
RecName = record_args['RecName']
pypath = fldtkn_args['pypath']
# print(pypath)
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
path = fldtkn_args['pypath']
full_path = os.path.join(WORKSPACE_PATH, path)
display(HTML(f'{path} <a href="{full_path}" target="_blank">Open File</a>'))

In [ ]:
module = load_module_variables(pypath)

# tokenizer_fn
tokenizer_fn = module.MetaDict['tokenizer_fn']

# idx2tkn
idx2tkn = module.MetaDict['idx2tkn']

if 'column_to_top_values' in module.MetaDict:
    fldtkn_args['column_to_top_values'] = module.MetaDict['column_to_top_values']
if 'item_to_configs' in module.MetaDict:
    fldtkn_args['item_to_configs'] = module.MetaDict['item_to_configs']

# Application

In [ ]:
from recfldtkn.pipeline_record import tokenizer_dfHumanRecAttr
from datetime import datetime

print('s', datetime.now())
RootID, RecID = cohort_args['RootID'], record_args['RecID']
df_fld = tokenizer_dfHumanRecAttr(dfHumanRecAttr, RootID, RecID, FldTknName, 
                                 tokenizer_fn, Vocab, fldtkn_args,
                                 use_tknidx = True)
print('e', datetime.now())
total_memory = df_fld.memory_usage(index=True).sum()
print(f"Total memory usage: {total_memory / 1024**2:.2f} MB")
df_fld.head()

In [ ]:
print('s', datetime.now())
RootID, RecID = cohort_args['RootID'], record_args['RecID']
df_fld = tokenizer_dfHumanRecAttr(dfHumanRecAttr, RootID, RecID, FldTknName, 
                                 tokenizer_fn, Vocab, fldtkn_args, 
                                 use_tknidx = False)
print('e', datetime.now())
total_memory = df_fld.memory_usage(index=True).sum()
print(f"Total memory usage: {total_memory / 1024**2:.2f} MB")
df_fld.head()